In [1]:
from numpy import *
from matplotlib.pyplot import *
newparams = {'figure.figsize': (8.0, 4.0), 'axes.grid': True,
             'lines.markersize': 8, 'lines.linewidth': 2,
             'font.size': 14}
rcParams.update(newparams)

### Oppgave 1

In [2]:
def evaluate(coeffs, x_data, reversed=False):
    """
    Horner's method for evaulating polynomials
        coeffs : List of coefficients from a_n to a_0
        x_data : Singleton, or numpy array of x-values for evaluation.
    Polynomial evaluated at x_data, either as singleton or numpy array.
    """

    # In case input coefficients is in reverse order
    if reversed:
        coeffs = coeffs[::-1]

    # Horner's method
    result = coeffs[0]
    for c in coeffs[1:]:
        result = (result * x_data) + c
    return result

def div_diff(x_data, y_data):
    """
    Creates a table of divided differences, to be used for netwon interpolation.
        x_data : Array of x-values.
        y_data : Array of y-values.
    2d-Array of divided differences.
    """

    # Initialize table for storing divided differences
    n = len(x_data)
    D = zeros((n, n))

    # Fill first column of table with y-values.
    D[:, 0] = y_data

    # Calculate differences
    for col in range(1, n):
        for row in range(n - col):
            D[row][col] = (D[row + 1][col - 1] - D[row][col - 1]) / (x_data[row + col] - x_data[row])

    return D

def newton_interpolation(x_data, y_data):
    """
    Implementation of Newton Interpolation.
        x_data : Array of x-values.
        y_data : Array of y-values.
    Interpolation polynomial, represented by a list of coefficients.
    """

    # Table of divided differences
    D = div_diff(x_data, y_data)
    coeffs = D[0]

    # Polynomial represented as coefficients
    p = zeros(len(x_data))

    # (x - x0)(x - x1) ... (x - xn)
    x_poly = array([1])

    # Construct polynomial by summing smaller polynomials
    for i, c in enumerate(coeffs):
        # p_n = c * (x - x_0)(x - x_1)...(x - x_[n-1]) + p_[n-1]
        p[-i - 1:] += c * x_poly

        # Update x_poly (x - x0)(x - x1) ... (x - xn)
        x_shift = append(x_poly, 0)
        x_scale = insert(x_poly, 0, 0) * x_data[i]
        x_poly = x_shift - x_scale

    return p

In [3]:
def f(x):
    return x**2 - 3

# Used for calculating error
exact = 3**0.5

results = []

for n in [2, 4, 8, 16]:
    x_data = linspace(1, 3, n + 1)
    y_data = f(x_data)
    
    # Swap x_data and y_data in order to find the inverse interpolation.
    p = newton_interpolation(y_data, x_data)
    
    # Approximated root of f(x) with corresponding error
    x_0 = evaluate(p, 0)
    e_0 = exact - x_0
    
    results.append((n, x_0, e_0))
    
    
# Printing
header = "|{:^8}|{:^20}|{:^20}|".format("Steps", "Approx.", "Error")
print(header)
print('='*len(header))
for result in results:
    print("|{:^8}|{:^20.8f}|{:^20.4e}|".format(*result))

| Steps  |      Approx.       |       Error        |
|   2    |     1.70000000     |     3.2051e-02     |
|   4    |     1.73463203     |    -2.5812e-03     |
|   8    |     1.73204590     |     4.9038e-06     |
|   16   |     1.73205081     |    -1.7032e-09     |


### Oppgave 3

In [4]:
def simpson(f, a, b, m=10):
    # Find an approximation to an integral by the composite Simpson's method:
    # Input:  
    #   f:    integrand
    #   a, b: integration interval
    #   m:    number of subintervals
    # Output: The approximation to the integral
    n = 2*m
    x_noder = linspace(a, b, n+1)       # equidistributed nodes from a to b 
    h = (b-a)/n                         # stepsize
    S1 = f(x_noder[0]) + f(x_noder[n])  # S1 = f(x_0)+f(x_n)
    S2 = sum(f(x_noder[1:n:2]))         # S2 = f(x_1)+f(x_3)+...+f(x_m)
    S3 = sum(f(x_noder[2:n-1:2]))       # S3 = f(x_2)+f(x_4)+...+f(x_{m-1})
    S = h*(S1 + 4*S2 + 2*S3)/3
    return S

In [5]:
def f(x):
    return exp(-x)

m = 26
a, b = 1, 3

approx = simpson(f, a, b, m)
exact = exp(-1) - exp(-3)

print(f"Approx: {approx}\tExact: {exact},\tError: {exact-approx}")

Approx: 0.3180923766700149	Exact: 0.3180923728035784,	Error: -3.866436504296189e-09


### Oppgave 4

In [6]:
def gauss_1(f, a, b):
    h = (b - a)/2
    c = (b + a)/2
    
    q_1 = f(c - sqrt(3/5)*h)
    q_2 = f(c)
    q_3 = f(c + sqrt(3/5)*h)
    
    return (h/9) * (5*q_1 + 8*q_2 + 5*q_3)

def gauss_basic(f, a, b):
    """
    Gauss–Legendre's quadrature with error estimate
        f : integrand
        a : lower integration limit
        b : upper integration limit
    Q_2(a, b) and the error estimate.
    """
    
    c = (b + a) / 2
    Q_1 = gauss_1(f, a, b)
    Q_2 = gauss_1(f, a, c) + gauss_1(f, c, b) 
    
    error_estimate = 1/63 * (Q_2 - Q_1)
    return Q_2, error_estimate

In [7]:
poly = lambda t: lambda x: x**t

a, b = -1, 1
results = []

for n in range(10):
    # Creates a polynomial of the form x**n
    f = poly(n)
    
    # Quadrature
    Q, _ = gauss_basic(f, a, b)
    
    # Exact result
    I_func = lambda x: poly(n+1)(x) / (n+1) # Expression for integral of polynomial
    I = I_func(b) - I_func(a)
    
    # Error
    E = I - Q
    if abs(E) < 1.0e-12:
        E = 0
    
    results.append((n, Q, I, E))
    
# Printing
header = "|{:^8}|{:^20}|{:^20}|{:^20}|".format("Degree", "Quadrature", "Integral", "Error")
print(header)
print('='*len(header))
for result in results:
    print("|{:^8}|{:^20.4e}|{:^20.4e}|{:^20.4e}|".format(*result))

| Degree |     Quadrature     |      Integral      |       Error        |
|   0    |     2.0000e+00     |     2.0000e+00     |     0.0000e+00     |
|   1    |     0.0000e+00     |     0.0000e+00     |     0.0000e+00     |
|   2    |     6.6667e-01     |     6.6667e-01     |     0.0000e+00     |
|   3    |     0.0000e+00     |     0.0000e+00     |     0.0000e+00     |
|   4    |     4.0000e-01     |     4.0000e-01     |     0.0000e+00     |
|   5    |     0.0000e+00     |     0.0000e+00     |     0.0000e+00     |
|   6    |     2.8500e-01     |     2.8571e-01     |     7.1429e-04     |
|   7    |     0.0000e+00     |     0.0000e+00     |     0.0000e+00     |
|   8    |     2.1692e-01     |     2.2222e-01     |     5.3056e-03     |
|   9    |     0.0000e+00     |     0.0000e+00     |     0.0000e+00     |


In [8]:
def gauss_adaptive(f, a, b, tol = 1.e-6, level = 0, maks_level=100):
    # Gauss's adaptive method
    # Input:  
    #   f:    integrand
    #   a, b: integration interval
    #   tol:  tolerance
    #   level, maks_level: For the recursion. Just ignore them. 
    # Output:
    #   The approximation to the integral
    
    Q, error_estimate = gauss_basic(f, a, b)    # The quadrature and the error estimate 

    if level >= maks_level:
        print('Warning: Maximum number of levels used.')
        return Q
    
    if abs(error_estimate) < tol:         # Accept the result, and return
        result = Q + error_estimate      
    else:
        # Divide the interval in two, and apply the algorithm to each interval.
        c = 0.5*(b+a)
        result_left  = gauss_adaptive(f, a, c, tol = 0.5*tol, level = level+1)
        result_right = gauss_adaptive(f, c, b, tol = 0.5*tol, level = level+1)
        result = result_right + result_left
    return result

In [9]:
f = lambda x: exp(-x)
g = lambda x: 1 / (1+16*x**2)
h = lambda x: (1 / ((x - 0.3)**2 + 0.01)) + (1 / ((x - 0.9)**2 + 0.04))

f_exact = exp(-1) - exp(-3)
g_exact = 0.25 * (arctan(20) - arctan(0))
h_exact = 41.326213804391148551

funcs = {}               
funcs['i'] = (f, f_exact, 1, 3)
funcs['ii'] = (g, g_exact, 0, 5)
funcs['iii'] = (h, h_exact, 0, 2)

header = "|{:^20}|{:^20}|{:^20}|{:^20}|{:^20}|".format("Function Name", "Quadrature", "Exact", "Error", "Tolerance")           
print(header) 
print('='*len(header))

for func_name, func_properties in funcs.items():
    func, func_exact, a, b = func_properties
    for tol in [1e-3, 1e-6, 1e-10]:
        Q = gauss_adaptive(func, a, b, tol)
        I = func_exact
        E = Q - I
        print("|{:^20}|{:^20.6e}|{:^20.6e}|{:^20.6e}|{:^20.6e}|".format(func_name, Q, I, E, tol))

|   Function Name    |     Quadrature     |       Exact        |       Error        |     Tolerance      |
|         i          |    3.180924e-01    |    3.180924e-01    |   -1.436156e-08    |    1.000000e-03    |
|         i          |    3.180924e-01    |    3.180924e-01    |   -1.436156e-08    |    1.000000e-06    |
|         i          |    3.180924e-01    |    3.180924e-01    |   -2.475797e-13    |    1.000000e-10    |
|         ii         |    3.941904e-01    |    3.802095e-01    |    1.398092e-02    |    1.000000e-03    |
|         ii         |    3.802094e-01    |    3.802095e-01    |   -1.044586e-07    |    1.000000e-06    |
|         ii         |    3.802095e-01    |    3.802095e-01    |   -4.870548e-13    |    1.000000e-10    |
|        iii         |    4.133569e+01    |    4.132621e+01    |    9.475766e-03    |    1.000000e-03    |
|        iii         |    4.132621e+01    |    4.132621e+01    |   -1.387976e-07    |    1.000000e-06    |
|        iii         |    4.132621e+0